In [1]:
import numpy as np
import pandas as pd
import ta
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

2023-06-09 16:49:27.643284: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-09 16:49:27.644619: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 16:49:27.670942: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 16:49:27.671623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 16:49:28.197280: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
#Import your data here
df = pd.read_csv('../raw_data/clean_raw_data.csv', delimiter=",")
df

,timestamp,open,high,low,close,volume,CMF,FII,MFI,VWA,...,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-17 04:00:00,10216.35,10238.95,10194.39,10209.45,872.764,-0.120924,-913.266701,28.652105,10185.127432,...,10275.790895,3005.70,2.25,1.70,1,0.45992,0.46011,-1,48.386232,-1
1,2019-09-17 05:00:00,10208.81,10232.07,10204.53,10223.97,819.923,-0.114688,917.954536,36.501645,10187.270366,...,10274.443495,3005.70,2.25,1.70,-1,0.40934,0.45992,-1,42.873022,-1
2,2019-09-17 06:00:00,10224.11,10230.44,10187.88,10224.09,922.804,-0.112046,802.637671,36.306243,10191.939033,...,10273.130448,3005.70,2.25,1.70,-1,0.31251,0.40934,-1,42.291553,-1
3,2019-09-17 07:00:00,10225.02,10230.79,10192.69,10193.85,905.598,-0.124361,-3224.208213,35.631136,10195.919801,...,10271.256418,3005.70,2.25,1.70,-1,0.14629,0.31251,-1,41.595938,-1
4,2019-09-17 08:00:00,10193.21,10215.07,10165.00,10195.82,1089.829,-0.114965,-2456.898021,34.774165,10199.535893,...,10269.459713,3005.70,2.25,1.70,-1,-0.27719,0.14629,-1,41.970769,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32551,2023-06-03 20:00:00,27239.40,27246.40,27101.00,27147.60,15130.376,-0.048901,45306.477416,59.148253,27189.586323,...,27073.676970,4282.37,5.25,5.08,1,0.55827,0.61369,-1,53.765071,1
32552,2023-06-03 21:00:00,27147.70,27173.70,27113.90,27138.80,6229.878,-0.075151,31002.276871,58.559862,27189.451257,...,27073.819644,4282.37,5.25,5.08,-1,0.15512,0.55827,-1,54.075485,1
32553,2023-06-03 22:00:00,27138.90,27153.60,27000.00,27084.90,15389.644,-0.042819,-91926.878625,49.242500,27175.830184,...,27072.988198,4282.37,5.25,5.08,-1,-0.36126,0.15512,-1,44.950464,1
32554,2023-06-03 23:00:00,27084.90,27119.50,27042.00,27052.70,7760.747,-0.045307,-114493.903593,43.575625,27168.759575,...,27071.634000,4282.37,5.25,5.08,-1,-0.86198,-0.36126,-1,45.530993,1


In [3]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

,timestamp,open,high,low,close,volume,CMF,FII,MFI,VWA,...,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-17 04:00:00,10216.35,10238.95,10194.39,10209.45,872.764,-0.120924,-913.266701,28.652105,10185.127432,...,10275.790895,3005.70,2.25,1.70,1,0.45992,0.46011,-1,48.386232,-1
1,2019-09-17 05:00:00,10208.81,10232.07,10204.53,10223.97,819.923,-0.114688,917.954536,36.501645,10187.270366,...,10274.443495,3005.70,2.25,1.70,-1,0.40934,0.45992,-1,42.873022,-1
2,2019-09-17 06:00:00,10224.11,10230.44,10187.88,10224.09,922.804,-0.112046,802.637671,36.306243,10191.939033,...,10273.130448,3005.70,2.25,1.70,-1,0.31251,0.40934,-1,42.291553,-1
3,2019-09-17 07:00:00,10225.02,10230.79,10192.69,10193.85,905.598,-0.124361,-3224.208213,35.631136,10195.919801,...,10271.256418,3005.70,2.25,1.70,-1,0.14629,0.31251,-1,41.595938,-1
4,2019-09-17 08:00:00,10193.21,10215.07,10165.00,10195.82,1089.829,-0.114965,-2456.898021,34.774165,10199.535893,...,10269.459713,3005.70,2.25,1.70,-1,-0.27719,0.14629,-1,41.970769,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32551,2023-06-03 20:00:00,27239.40,27246.40,27101.00,27147.60,15130.376,-0.048901,45306.477416,59.148253,27189.586323,...,27073.676970,4282.37,5.25,5.08,1,0.55827,0.61369,-1,53.765071,1
32552,2023-06-03 21:00:00,27147.70,27173.70,27113.90,27138.80,6229.878,-0.075151,31002.276871,58.559862,27189.451257,...,27073.819644,4282.37,5.25,5.08,-1,0.15512,0.55827,-1,54.075485,1
32553,2023-06-03 22:00:00,27138.90,27153.60,27000.00,27084.90,15389.644,-0.042819,-91926.878625,49.242500,27175.830184,...,27072.988198,4282.37,5.25,5.08,-1,-0.36126,0.15512,-1,44.950464,1
32554,2023-06-03 23:00:00,27084.90,27119.50,27042.00,27052.70,7760.747,-0.045307,-114493.903593,43.575625,27168.759575,...,27071.634000,4282.37,5.25,5.08,-1,-0.86198,-0.36126,-1,45.530993,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32556 entries, 0 to 32555
Data columns (total 78 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   timestamp               32556 non-null  datetime64[ns]
 1   open                    32556 non-null  float64       
 2   high                    32556 non-null  float64       
 3   low                     32556 non-null  float64       
 4   close                   32556 non-null  float64       
 5   volume                  32556 non-null  float64       
 6   CMF                     32556 non-null  float64       
 7   FII                     32556 non-null  float64       
 8   MFI                     32556 non-null  float64       
 9   VWA                     32556 non-null  float64       
 10  AOI                     32556 non-null  float64       
 11  KAMA                    32556 non-null  float64       
 12  PPO                     32556 non-null  float6

In [5]:
# Select the relevant columns for modeling
df_selected = df.drop(['close'], axis=1)

In [6]:
# Convert the timestamp column to datetime if it's not already
df_selected['timestamp'] = pd.to_datetime(df_selected['timestamp'])

In [7]:
# Split the data into features (X) and target (y)
X = df_selected.drop('timestamp', axis=1).values
y = df['close'].values

In [8]:
y

array([10209.45, 10223.97, 10224.09, ..., 27084.9 , 27052.7 , 26990.  ])

In [9]:
X

array([[ 1.02163500e+04,  1.02389500e+04,  1.01943900e+04, ...,
        -1.00000000e+00,  4.83862317e+01, -1.00000000e+00],
       [ 1.02088100e+04,  1.02320700e+04,  1.02045300e+04, ...,
        -1.00000000e+00,  4.28730216e+01, -1.00000000e+00],
       [ 1.02241100e+04,  1.02304400e+04,  1.01878800e+04, ...,
        -1.00000000e+00,  4.22915526e+01, -1.00000000e+00],
       ...,
       [ 2.71389000e+04,  2.71536000e+04,  2.70000000e+04, ...,
        -1.00000000e+00,  4.49504636e+01,  1.00000000e+00],
       [ 2.70849000e+04,  2.71195000e+04,  2.70420000e+04, ...,
        -1.00000000e+00,  4.55309932e+01,  1.00000000e+00],
       [ 2.70528000e+04,  2.70693000e+04,  2.69000000e+04, ...,
        -1.00000000e+00,  3.99181007e+01, -1.00000000e+00]])

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [11]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
X_train_scaled.shape

(26044, 76)

In [13]:
# Reshape the data for LSTM input (samples, time steps, features)
n_features = X_train_scaled.shape[1]
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, n_features))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, n_features))

In [14]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(1, n_features)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

2023-06-09 16:49:29.165678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-09 16:49:29.166021: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-09 16:49:29.331647: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/conca

In [18]:
# Train the model
model.fit(X_train_reshaped, y_train, epochs=250, batch_size=32)


Epoch 1/250
814/814 [==============================] - 1s 1ms/step - loss: 833288640.0000
Epoch 2/250
814/814 [==============================] - 1s 1ms/step - loss: 831487616.0000
Epoch 3/250
814/814 [==============================] - 1s 1ms/step - loss: 829688960.0000
Epoch 4/250
814/814 [==============================] - 1s 1ms/step - loss: 827893760.0000
Epoch 5/250
814/814 [==============================] - 1s 1ms/step - loss: 826102848.0000
Epoch 6/250
814/814 [==============================] - 1s 1ms/step - loss: 824314816.0000
Epoch 7/250
814/814 [==============================] - 1s 1ms/step - loss: 822527104.0000
Epoch 8/250
814/814 [==============================] - 1s 1ms/step - loss: 820742976.0000
Epoch 9/250
814/814 [==============================] - 1s 1ms/step - loss: 818961536.0000
Epoch 10/250
814/814 [==============================] - 1s 1ms/step - loss: 817186816.0000
Epoch 11/250
814/814 [==============================] - 1s 1ms/step - loss: 815416192.0000
Epoch 12

In [ ]:
# Evaluate the model on the test data
loss = model.evaluate(X_test_reshaped, y_test)
loss

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test_reshaped)


In [ ]:
predictions = scaler.inverse_transform(y_pred)

In [ ]:
# Plot the actual values
val.plot(label='Actual')

# Plot the predicted values
preds.plot(label='Predicted')

# Set plot labels and title
plt.xlabel('Timestamp')
plt.ylabel('Close Price')
plt.title('Actual vs Predicted')

# Display the legend
plt.legend()

# Show the plot
plt.show()